In [1]:
#https://github.com/backstopmedia/tensorflowbook/blob/master/chapters/04_machine_learning_basics/linear_regression.py
#generic.py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
#模型参数
W = tf.Variable(tf.zeros([2,1]),name="weights") #权重
b = tf.Variable(0.,name="bias") #偏置
# print(W)
# print(b)

In [4]:
def inputs():
    """读取训练数据 
    身高体重和高血脂的关系 http://people.sc.fsu.edu/~jburkardt/datasets/regression/x09.txt"""
    #对比 get_started.ipynb 文件中的输入值？
    weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], [63, 28], [72, 36], [79, 57], [75, 44], [27, 24], [89, 31], [65, 52], [57, 23], [59, 60], [69, 48], [60, 34], [79, 51], [75, 50], [82, 34], [59, 46], [67, 23], [85, 37], [55, 40], [63, 30]]
    blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 365, 209, 290, 346, 254, 395, 434, 220, 374, 308, 220, 311, 181, 274, 303, 244]
    
    #to_float ?
    return tf.to_float(weight_age), tf.to_float(blood_fat_content) 

In [5]:
def inference(X):
    """推断模型"""
    return tf.matmul(X,W) + b #matmul(),矩阵W,X相乘 y=wX+b

def loss(X,Y):
    """损失函数，训练数据X，期望输出Y，计算损失
    总平方误差，预测的输出向量与期望输出之差的平方和，也叫 L2范数 或 L2损失函数
    """
    # Y_tmp = inference(X)
    # Y_predicted = tf.transpose(Y_tmp) #转置
    # tmp = tf.squared_difference(Y,Y_predicted) #平方差
    # return tf.reduce_sum(tmp) #总平方误差
    
    Y_predicted = tf.transpose(inference(X)) # make it a row vector
    return tf.reduce_sum(tf.squared_difference(Y, Y_predicted)) 

def train(total_loss):
    """依据 计算的总损失 训练或调整 模型参数"""
    learning_rate = 0.000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss) #梯度下降算法

def evaluate(sess,X,Y):
    """对训练得到的模型进行评估"""
    print("50,20 ", sess.run(inference([[50., 20.]]))) # ~ 303
    print("50,70 ",sess.run(inference([[50., 70.]]))) # ~ 256
    print("90,20 ",sess.run(inference([[90., 20.]]))) # ~ 303
    print("90,70 ",sess.run(inference([[90., 70.]]))) # ~ 256
    

In [6]:
saver = tf.train.Saver() #设置检查点

checkpoints_files="./tmp/checkpoints/ml_linear_regression"

with tf.Session() as sess:
    # tf.initialize_all_variables().run() #ver0.9
    tf.global_variables_initializer().run() #ver1.5
    

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss) #节点依赖

    coord = tf.train.Coordinator() #多线程调度
    threads = tf.train.start_queue_runners(sess=sess, coord=coord) #?

    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op]) #[] ?
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:  
            print("Epoch:%s, loss:%s"%(step,sess.run(total_loss)))
            saver.save(sess,checkpoints_files,global_step=step)
            
    print( "Final model W=", sess.run(W), "b=", sess.run(b) )
    evaluate(sess, X, Y) 
    
    coord.request_stop()
    coord.join(threads)
    
    #~ tensorboard --logdir="./tmp/graph/ml_linear_regression"
    writer = tf.summary.FileWriter('./tmp/graph/ml_linear_regression',sess.graph)
    writer.close()
    
    saver.save(sess,checkpoints_files,global_step=training_steps) 
    sess.close() #使用with还需要close？
    
    #3D绘图？
#     plt.scatter(tf.transpose(X[:0]),tf.transpose(X[:1]))
#     plt.show()
    
       

Epoch:0, loss:1230281.8
Epoch:10, loss:92727.65
Epoch:20, loss:87290.99
Epoch:30, loss:82880.48
Epoch:40, loss:78954.15
Epoch:50, loss:75458.734
Epoch:60, loss:72346.89
Epoch:70, loss:69576.59
Epoch:80, loss:67110.29
Epoch:90, loss:64914.66
Epoch:100, loss:62959.95
Epoch:110, loss:61219.773
Epoch:120, loss:59670.57
Epoch:130, loss:58291.363
Epoch:140, loss:57063.504
Epoch:150, loss:55970.41
Epoch:160, loss:54997.25
Epoch:170, loss:54130.867
Epoch:180, loss:53359.566
Epoch:190, loss:52672.902
Epoch:200, loss:52061.59
Epoch:210, loss:51517.348
Epoch:220, loss:51032.82
Epoch:230, loss:50601.47
Epoch:240, loss:50217.414
Epoch:250, loss:49875.527
Epoch:260, loss:49571.137
Epoch:270, loss:49300.133
Epoch:280, loss:49058.867
Epoch:290, loss:48844.05
Epoch:300, loss:48652.82
Epoch:310, loss:48482.547
Epoch:320, loss:48330.945
Epoch:330, loss:48195.973
Epoch:340, loss:48075.793
Epoch:350, loss:47968.8
Epoch:360, loss:47873.535
Epoch:370, loss:47788.703
Epoch:380, loss:47713.184
Epoch:390, loss: